In [1]:
import logging


In [2]:
from dotenv import load_dotenv
import os
#import json

In [3]:
from util import run_command
from titan_classes import get_files_titan_xml, convert_gmt_utc, build_record_titan_df, make_at_time, clean_string
from titan_classes import TITAN

In [4]:
# Load environment variables from the .env file
load_dotenv()

# Access environment variables
TUNER_ID = os.getenv('TUNER_ID')
TUNER_IP = os.getenv('TUNER_IP')

TUNER_ID, TUNER_IP

('10B16BC7', '192.168.0.151')

In [5]:
RECORDINGS = os.getenv('DVR_RECORDINGS')
RECORDINGS

'/home/tony/Videos'

In [6]:
DVR_HOME = '/home/tony/DVR'

# LOGGING

In [7]:
# Configure logging
logging.basicConfig(
    filename="/home/tony/DVR/log/dvr.log", 
    level=logging.INFO, 
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Create a logger
logger = logging.getLogger(__name__)

# Log some messages
logger.info("DVR application started.")
# logger.warning("Low disk space detected.")
# logger.error("Motion sensor malfunction.")

In [8]:
to_be_recorded = get_files_titan_xml()
to_be_recorded

['/home/tony/Downloads/record-program(4).tvpi',
 '/home/tony/Downloads/record-program.tvpi',
 '/home/tony/Downloads/record-program(1).tvpi',
 '/home/tony/Downloads/record-program(5).tvpi',
 '/home/tony/Downloads/record-program(2).tvpi',
 '/home/tony/Downloads/record-program(3).tvpi']

In [9]:
program_todo_list = build_record_titan_df(to_be_recorded)

20250114 01:15
20250111 23:30
20250112 00:00
20250112 21:30
20250112 01:00
20250112 15:00


In [10]:
#program_todo_list

In [11]:
def duration_to_hms(duration_str):
  """
  Converts a duration string (e.g., "02:45") to hours, minutes, and seconds.

  Args:
    duration_str: The duration string in the format "HH:MM" or "MM:SS".

  Returns:
    A tuple containing hours, minutes, and seconds as integers.
  """
  try:
    if ':' not in duration_str:
      raise ValueError("Invalid duration format. Expected 'HH:MM' or 'MM:SS'.")

    parts = duration_str.split(':')
    if len(parts) == 2:
      hours = int(parts[0]) if parts[0] else 0
      minutes = int(parts[1])
      seconds = 0
    elif len(parts) == 1:
      hours = 0
      minutes = int(parts[0])
      seconds = 0
    else:
      raise ValueError("Invalid duration format. Expected 'HH:MM' or 'MM:SS'.")

    seconds = (hours * 60 + minutes) * 60

    return hours, minutes, seconds

  except ValueError as e:
    print(f"Error: {e}")
    return None

# Example usage:
duration = "02:45"
hours, minutes, seconds = duration_to_hms(duration)

if hours is not None:
  print(f"Hours: {hours}")
  print(f"Minutes: {minutes}")
  print(f"Seconds: {seconds}")

Hours: 2
Minutes: 45
Seconds: 9900


In [12]:
def duration(dur):
    hours, minutes, seconds = duration_to_hms(dur)
    if seconds > 3600 * 2 - 1:
        seconds += 3600 + 1800
    return seconds

In [13]:
def get_vchan(a):
    chan = a['psip-major']
    minor = a['psip-minor']
    vchan = f'{chan}.{minor}'
    return vchan

In [14]:
def run_at(name, time, dur_seconds, vchan):
    global RECORDINGS
    global TUNER_IP
    cmd=f'echo curl -so {RECORDINGS}/{name} {TUNER_IP}:5004/auto/{vchan}?duration={dur_seconds}| at -t {time}'
    logger.info(cmd)
    print(cmd)
    run_command(cmd)
    

In [15]:
def schedule_record(a):
    station = a['station']
    ptitle = a['program-title']
    #etitle = a['episode-title']
    etitle = a.get('episode-title', 'One') 
    at_time = make_at_time(a['start-date'], a['start-time'])
                
    name_of_recording = f'{station}_{ptitle}_{etitle}_{at_time}.ts'
    clean_name_of_recording = clean_string(name_of_recording)
    print(clean_name_of_recording)
    # process duration
    dur = a['duration']
    print(dur)
    dur_seconds = duration(dur)
    print(dur_seconds)
    print(at_time)
    vchan = get_vchan(a)
    print(vchan)
    run_at(clean_name_of_recording, at_time, dur_seconds, vchan)

In [16]:
program_todo_list[0]

{'station': 'KSFY-DT',
 'tv-mode': 'digital',
 'program-title': 'Monday Night Football',
 'program-description': 'Debuting in 1970, Monday Night Football caps off the week in the NFL with one of the more premier matchups in the league, featuring games between elite teams.',
 'start-date': '20250114',
 'start-time': '01:15',
 'end-date': '20250114',
 'end-time': '04:00',
 'duration': '02:45',
 'rf-channel': '13',
 'psip-major': '13',
 'psip-minor': '1',
 'stream-number': '1'}

In [17]:
for show in program_todo_list:
    schedule_record(show)

KSFY_DT_Monday_Night_Football_One_202501131915_ts
02:45
15300
202501131915
13.1
echo curl -so /home/tony/Videos/KSFY_DT_Monday_Night_Football_One_202501131915_ts 192.168.0.151:5004/auto/13.1?duration=15300| at -t 202501131915
KDLT_DT_NBC_Nightly_News_with_Lester_Holt_One_202501111730_ts
00:30
1800
202501111730
46.1
echo curl -so /home/tony/Videos/KDLT_DT_NBC_Nightly_News_with_Lester_Holt_One_202501111730_ts 192.168.0.151:5004/auto/46.1?duration=1800| at -t 202501111730
KDLT_DT_Dakota_News_Now_Saturday___6pm_on_KDLT_One_202501111800_ts
00:30
1800
202501111800
46.1
echo curl -so /home/tony/Videos/KDLT_DT_Dakota_News_Now_Saturday___6pm_on_KDLT_One_202501111800_ts 192.168.0.151:5004/auto/46.1?duration=1800| at -t 202501111800
KDLT_FOX_NFL_Playoffs_NFC_Wild_Card_Game__Green_Bay_Packers_at_Philadelphia_Eagles_from_Lincoln_Financial_Field_202501121530_ts
03:00
16200
202501121530
46.2
echo curl -so /home/tony/Videos/KDLT_FOX_NFL_Playoffs_NFC_Wild_Card_Game__Green_Bay_Packers_at_Philadelphia_Ea

In [21]:
# clean up tvpi files
for xml_file in to_be_recorded:
    print('deleteing:', xml_file)
    os.remove(xml_file)

deleteing: /home/tony/Downloads/record-program(4).tvpi
deleteing: /home/tony/Downloads/record-program.tvpi
deleteing: /home/tony/Downloads/record-program(1).tvpi
deleteing: /home/tony/Downloads/record-program(5).tvpi
deleteing: /home/tony/Downloads/record-program(2).tvpi
deleteing: /home/tony/Downloads/record-program(3).tvpi


In [18]:
#cmd=f'echo curl -o {RECORDINGS}/tony1nbc.ts {TUNER_IP}:5004/auto/v46.1?duration=3600 | at 13:25'


In [19]:
#! echo curl -so /home/tony/Videos/KSFY_DT_Monday_Night_Football_One_202501131915_ts 192.168.0.151:5004/auto/13.1?duration=15300| at 202501131915